# Multiple Linear Regression

## Introduction

Assume that we have a labelled dataset with $m$ samples, continuous features $X_1,\dotsc,X_n$, and continuous target $Y$. We denote the feature columns by $\mathbf{x}_i \in \mathbb{R}^m$, and the target column by $\mathbf{y} \in \mathbb{R}^m$. 

### The MLE model
The multiple linear regression model assumes that the target is a linear combination of the features, plus some noise. That is, it assumes that there exists some linear function
\begin{equation*}
    F_{b,\mathbf{w}}(X_1,\dotsc,X_n) = b + w_1X_1 + \dotsb + w_nX_n,
\end{equation*}
where $\mathbf{w} = (w_1,\dotsc,w_n) \in \mathbb{R}^n$ and $b \in \mathbb{R}$, such that
\begin{equation*}
    y_i = F_{\mathbf{w}}(x_{i1},\dotsc,x_{in}) + \epsilon_i, \quad \textup{ for } i=1,\dotsc,m.
\end{equation*}
Above, each $\epsilon_i$ is a random variable representing the noise, $y_i$ is the target value for the $i$-th sample, and $x_{ij}$ is the $j$-th feature value for the $i$-th sample. The parameter space for this model consists of all vectors $(b,\mathbf{w}) \in \mathbb{R}^{n+1}$. The $w_i$'s are called *weights*, and $b$ is called the *bias* (of the model).

As usual, we denote the predicted values of the target by
\begin{equation*}
    \hat{y}_i = F_{\mathbf{w}}(x_{i1},\dotsc,x_{in}), \quad \textup{ for } i=1,\dotsc,m,
\end{equation*}
and we collect these all into a single vector of predictions $\hat{\mathbf{y}} = \begin{bmatrix} \; \hat{y}_1 & \dotsb & \hat{y}_n \; \end{bmatrix} \in \mathbb{R}^m$. There are three equivalent ways to express the vector $\hat{\mathbf{y}}$ in terms of the features and weights:

1. *As a linear combination of the features*: In this case, we write 
\begin{equation*}
    \hat{\mathbf{y}} = b \mathbf{1} + w_1 \mathbf{x}_1 + \dotsb + w_n \mathbf{x}_n \in \mathbb{R}^m.
\end{equation*}
In this case, the residuals are given by
\begin{equation*}
    \mathbf{y} - \hat{\mathbf{y}} = \mathbf{y} - (b \mathbf{1} + w_1 \mathbf{x}_1 + \dotsb + w_n \mathbf{x}_n) \in \mathbb{R}^m.
\end{equation*}
2. *As a matrix product plus a vector*: In this case, we write
\begin{equation*}
    \hat{\mathbf{y}} = b\mathbf{1} + \mathbf{X} \mathbf{w} \in \mathbb{R}^m,
\end{equation*}
where $\mathbf{X} \in \mathbb{R}^{m \times n}$ is the matrix of features (the "design matrix"), and $\mathbf{w} = (w_1,\dotsc,w_n) \in \mathbb{R}^n$ is the vector of weights. In this case, the residuals are given by
\begin{equation*}
    \mathbf{y} - \hat{\mathbf{y}} = \mathbf{y} - (b \mathbf{1} + \mathbf{X} \mathbf{w}) \in \mathbb{R}^m.
\end{equation*}
3. *As a matrix product*: In this case, we write
\begin{equation*}
    \hat{\mathbf{y}} = \overline{\mathbf{X}} \overline{\mathbf{w}} \in \mathbb{R}^m,
\end{equation*}
where $\overline{\mathbf{X}} \in \mathbb{R}^{m \times (n+1)}$ is the "augmented design matrix" (i.e. the original design matrix with an extra column of ones $\mathbf{1} \in \mathbb{R}^m$ added to the left), and $\overline{\mathbf{w}} = (b,\mathbf{w}) = (b,w_1,\dotsc,w_n) \in \mathbb{R}^{n+1}$ is the vector of bias and weights. In this case, the residuals are given by
\begin{equation*}
    \mathbf{y} - \hat{\mathbf{y}} = \mathbf{y} - \overline{\mathbf{X}} \overline{\mathbf{w}} \in \mathbb{R}^m.
\end{equation*}

### Formula for MSE loss
Similar to the single-variable case, we can use the mean squared error (MSE) as a loss function. The MSE in this case is given by
\begin{align*}
    J(\mathbf{w}) & = \frac{1}{m} || \mathbf{y} - \hat{\mathbf{y}} ||^2 \\
                & = \frac{1}{m} || \mathbf{y} - (b \mathbf{1} + w_1 \mathbf{x}_1 + \dotsb + w_n \mathbf{x}_n) ||^2.
\end{align*}
Fitting the models means we want to find the optimal parameters $(\hat{b},\hat{\mathbf{w}})$ that minimize the MSE:
\begin{equation*}
    (\hat{b},\hat{\mathbf{w}}) = \argmin_{(b,\mathbf{w})} J(b,\mathbf{w}).
\end{equation*}

### Strategy for finding optimal parameters
We will use linear algebra to derive the optimal parameters. The broad idea is the same as the single-variable case. That is, as $(b,\mathbf{w})$ varies over the parameter space $\mathbb{R}^{n+1}$, the vector $\hat{\mathbf{y}}$ varies over the subspace
\begin{equation*}
    U = \textup{span}(\mathbf{1},\mathbf{x}_1,\dotsc,\mathbf{x}_n) \subseteq \mathbb{R}^m.
\end{equation*}
The optimal parameters are found when $\hat{\mathbf{y}}$ is the closest point to $\mathbf{y}$ in $U$ (i.e. it is the linear combination of $\mathbf{1},\mathbf{x}_1,\dotsc,\mathbf{x}_n$ that is closest to $\mathbf{y}$). This is equivalent to saying that $\hat{\mathbf{y}}$ is the orthogonal projection of $\mathbf{y}$ onto $U$. By definition, this means that $\hat{\mathbf{y}}$ is the unique vector in $U$ such that
\begin{equation*}
    \mathbf{y} - \hat{\mathbf{y}} \textup{ is orthogonal to every vector in } U.
\end{equation*}
As you will prove in homework, this is the same as saying that
\begin{equation}\tag{1}
    \mathbf{y} - \hat{\mathbf{y}} \textup{ is orthogonal to } \mathbf{1},\mathbf{x}_1,\dotsc,\mathbf{x}_n.
\end{equation}
Now, we can proceed in two ways, to arrive at the same result:

1.  The first way is to use the augmented design matrix
\begin{equation*}
    \overline{\mathbf{X}} = \begin{bmatrix} \; \mathbf{1} & \mathbf{x}_1 & \dotsb & \mathbf{x}_n \; \end{bmatrix} \in \mathbb{R}^{m \times (n+1)},
\end{equation*}
and get a nice-looking matrix equation that needs solving. This method will require us to invert an $(n+1) \times (n+1)$ matrix. 
2. The second way is to deal separately with the bias term $b$ first, because it turns out to be rather easy to determine the optimal value of $b$. Then, we do some finessing of the feature vectors and arrive at a beautiful matrix equation that again needs solving. This method will require us to invert an $n \times n$ matrix.

## Method 1 for fitting: using the augmented design matrix
Recall that orthogonality is expressed by saying that the dot product is $0$, and recall also that the dot product of vectors $\mathbf{u}, \mathbf{v} \in \mathbb{R}^m$ can be expressed as matrix multiplication $\mathbf{u}^T \mathbf{v}$, where the first is a row vector and the second is a column vector. 

Thus, (1) is equivalent to saying that $(\hat{b},\hat{\mathbf{w}})$ satisfy the following $n+1$ equations:
\begin{align*}
    \mathbf{1}^T(\mathbf{y} - \hat{\mathbf{y}}) & = 0, \\
    \mathbf{x}_1^T(\mathbf{y} - \hat{\mathbf{y}}) & = 0, \\
    & \vdots \\
    \mathbf{x}_n^T(\mathbf{y} - \hat{\mathbf{y}}) & = 0.
\end{align*}
Re-writing this in matrix form, this is equivalent to saying that
\begin{equation*}
    \overline{X}^T (\mathbf{y} - \hat{\mathbf{y}}) = \begin{bmatrix} \mathbf{1}^T \\ \mathbf{x}_1^T \\ \vdots \\ \mathbf{x}_n^T \end{bmatrix} (\mathbf{y} - \hat{\mathbf{y}}) = 0.
\end{equation*}
If we write $\hat{\mathbf{y}}$ as $\overline{\mathbf{X}} \overline{\mathbf{w}}$, where $\overline{\mathbf{w}} = (b, \mathbf{w}) \in \mathbb{R}^{n+1}$, then the above equation can be re-arranged to give the following matrix equation:
\begin{equation}\tag{2}
    \overline{\mathbf{X}}^T \overline{\mathbf{X}} \overline{\mathbf{w}} = \overline{\mathbf{X}}^T \mathbf{y} = \begin{bmatrix} \mathbf{1} \cdot \mathbf{y} \\ \mathbf{x}_1 \cdot \mathbf{y} \\ \vdots \\ \mathbf{x}_n \cdot \mathbf{y} \end{bmatrix} \in \mathbb{R}^{n+1}. 
\end{equation}
Thus, our optimal parameters $(\hat{b},\hat{\mathbf{w}})$ are given by solving (4). To solve it, observe that 
$$\overline{\mathbf{X}}^T \overline{\mathbf{X}} = \begin{bmatrix}
\mathbf{1}\cdot \mathbf{1} & \mathbf{1} \cdot \mathbf{x}_1 & \dotsb & \mathbf{1} \cdot \mathbf{x}_n \\
\mathbf{x}_1 \cdot \mathbf{1} & \mathbf{x}_1 \cdot \mathbf{x}_1 & \dotsb & \mathbf{x}_1 \cdot \mathbf{x}_n \\
\vdots & \vdots & \ddots & \vdots \\
\mathbf{x}_n \cdot \mathbf{1} & \mathbf{x}_n \cdot \mathbf{x}_1 & \dotsb & \mathbf{x}_n \cdot \mathbf{x}_n
\end{bmatrix} \in \mathbb{R}^{(n+1) \times (n+1)}.$$
This square matrix is called the **Gram matrix** associated to the vectors $\mathbf{1},\mathbf{x}_1,\dotsc,\mathbf{x}_n$. It is of fundamental importance because: *the Gram matrix is invertible if and only if the vectors are linearly independent*. In practice, when the feature vectors arise from real-world data, the columns will almost certainly be linearly independent (otherwise, it would mean that one of our features is on-the-nose a linear combination of the others). Thus, it is safe to assume that the Gram matrix is invertible in our case. 

Now, we solve for $\overline{\mathbf{w}} = (b, \mathbf{w})$ in (4) by multiplying both sides of (4) by the inverse of the Gram matrix:
\begin{equation}\tag{3}
    \overline{\mathbf{w}} = (\overline{\mathbf{X}}^T \overline{\mathbf{X}})^{-1} \overline{\mathbf{X}}^T \mathbf{y} \in \mathbb{R}^{n+1}.
\end{equation}
Computing the right-hand side immediately gives us the optimal parameters $(\hat{b},\hat{\mathbf{w}}) \in \mathbb{R}^{n+1}$! This concludes method (1) for fitting the model.

## Method 2 for fitting: dealing with bias, then de-meaning

### Means and de-meaned vectors
Before moving on to method 2, it behooves us to recall some facts about means of vectors and de-meaned vectors. Recall that for any vector $\mathbf{v} = \begin{bmatrix} \; v_1 & \dotsb & v_m \; \end{bmatrix}^T \in \mathbb{R}^m$, we have
\begin{equation*}
    \mathbf{1} \cdot \mathbf{v} = \sum_{i=1}^m v_i = m \cdot \overline{v},
\end{equation*}
where $\overline{v} = \frac{1}{m} \sum_{i=1}^m v_i$ is the **mean** of the entries of $\mathbf{v}$. Recall from the previous notebook that we denote the **de-meaned** vector $\mathbf{v} - \overline{v} \mathbf{1}$ by $\mathbf{v}' \in \mathbb{R}^m$. Explicitly,
\begin{equation*}
    \mathbf{v}' = \begin{bmatrix} \; v_1 - \overline{v} & \dotsb & v_m - \overline{v} \; \end{bmatrix}^T \in \mathbb{R}^m.
\end{equation*}
It is called a "de-meaned" vector because its mean $\overline{\mathbf{v}'}$ equals $0$. You should think of $\overline{\mathbf{v}} \mathbf{1}$ as the "constant part" of $\mathbf{v}$. The entries of the de-meaned vector $\mathbf{v}'$ should be thought of as the "variations" of the entries of $\mathbf{v}$ from its mean. 

### Dealing with the bias term first
Again we start with the condition (1), i.e. that $\mathbf{y} - \hat{\mathbf{y}}$ should be orthogonal to $\mathbf{1},\mathbf{x}_1,\dotsc,\mathbf{x}_n$. Let us consider first what is imposed on the parameters when we require $\mathbf{y} - \hat{\mathbf{y}}$ to be orthogonal to $\mathbf{1}$. Being orthogonal to $\mathbf{1}$ means that
\begin{align*}
    0 & = \mathbf{1} \cdot (\mathbf{y} - \hat{\mathbf{y}}) \\
        & = \mathbf{1} \cdot (\mathbf{y} - (b \mathbf{1} + w_1 \mathbf{x}_1 + \dotsb + w_n \mathbf{x}_n)) \\
        & = \mathbf{1}\cdot \mathbf{y} - b (\mathbf{1} \cdot \mathbf{1}) - \sum_{i=1}^n w_i (\mathbf{1} \cdot \mathbf{x}_i) \\
        & = m \left( \overline{\mathbf{y}} - b - \sum_{i=1}^n w_i \overline{\mathbf{x}}_i \right).
\end{align*}
Dividing by $m$ and re-arranging, we find that $\mathbf{y} - \hat{\mathbf{y}}$ is orthogonal to $\mathbf{1}$ if and only if
\begin{equation}\tag{4}
    b = \overline{\mathbf{y}} - \sum_{i=1}^n w_i \overline{\mathbf{x}}_i.
\end{equation}
So, if we can find the optimal weights $\hat{\mathbf{w}} \in \mathbb{R}^n$, then we can find the optimal bias $\hat{b}$ using (4):
\begin{equation*}
    \hat{b} = \overline{\mathbf{y}} - \sum_{i=1}^n \hat{w}_i \overline{\mathbf{x}}_i.
\end{equation*}
The equation (4) admits a pleasant interpretation using the concept of "center of mass". That is, if we visualize each row (features plus target) as a point $(x_{i1},\dotsc,x_{in},y_i)$ in $\mathbb{R}^{n+1}$, then the labelled dataset can be visualized as a cloud of points in $\mathbb{R}^{n+1}$. Then, the point $(\overline{\mathbf{x}}_1,\dotsc, \overline{\mathbf{x}}_n,\overline{\mathbf{y}}) \in \mathbb{R}^{n+1}$ is the center of mass of this point-cloud. Condition (4) says that fitted model (more precisely, the graph of the fitted model) must pass through the center of mass of this point-cloud!

### Getting rid of the bias term
Using (4), we can get rid of $b$ in the formula for $\hat{\mathbf{y}}$:
\begin{align*}
    \hat{\mathbf{y}} & = b \mathbf{1} + w_1 \mathbf{x}_1 + \dotsb + w_n \mathbf{x}_n \\
                    & = \left( \overline{\mathbf{y}} - \sum_{i=1}^n w_i \overline{\mathbf{x}}_i \right) \mathbf{1} + \sum_{i=1}^n w_i \mathbf{x}_i\\
                    & = \overline{\mathbf{y}} \mathbf{1} - \sum_{i=1}^n w_i \overline{\mathbf{x}}_i \mathbf{1} + \sum_{i=1}^n w_i \mathbf{x}_i\\
                    & = \overline{\mathbf{y}} \mathbf{1} + \sum_{i=1}^n w_i (\mathbf{x}_i - \overline{\mathbf{x}}_i \mathbf{1}) \\
                    & = \overline{\mathbf{y}} \mathbf{1} + \sum_{i=1}^n w_i \mathbf{x}_i' \; \in \mathbb{R}^m.
\end{align*}
Next, we can get rid of $b$ in the formula for the residuals:
\begin{equation}\tag{5}
\begin{alignedat}{1}
    \mathbf{y} - \hat{\mathbf{y}} & = \mathbf{y} - \left( \overline{\mathbf{y}} \mathbf{1} + \sum_{i=1}^n w_i \mathbf{x}_i' \right) \\
                                & = \mathbf{y} - \overline{\mathbf{y}} \mathbf{1} - \sum_{i=1}^n w_i \mathbf{x}_i'\\
                                & = \mathbf{y}' - \sum_{i=1}^n w_i \mathbf{x}_i'\\
                                & = \mathbf{y}' - \mathbf{X}' \mathbf{w} \; \in \mathbb{R}^m,
\end{alignedat}
\end{equation}
where $\mathbf{X}' = \begin{bmatrix} \; \mathbf{x}_1' & \dotsb & \mathbf{x}_n' \; \end{bmatrix} \in \mathbb{R}^{m \times n}$ is the matrix of de-meaned features.

Thus, conditions (4) and (5) together imply that the optimal parameters $(\hat{b},\hat{\mathbf{w}})$ satisfy the following two conditions:

1. The optimal weights $\hat{\mathbf{w}} \in \mathbb{R}^n$ satisfy
\begin{equation}\tag{6}
\begin{alignedat}{1}
    \hat{\mathbf{w}} & = \argmin_{\mathbf{w} \in \mathbb{R}^n} || \mathbf{y}' - \mathbf{X}' \mathbf{w} ||^2\\
    & = \argmin_{\mathbf{w} \in \mathbb{R}^n} || \mathbf{y}' - (w_1 \mathbf{x}_1' + \dotsb + {w}_n \mathbf{x}_n)||^2.
\end{alignedat}
\end{equation}
In other words, taking a weighted average of the de-meaned feature vectors $\mathbf{x}_1',\dotsc,\mathbf{x}_n'$ should bring us as close to the de-meaned target vector $\mathbf{y}'$ as possible. In other words, we want to approximate how $\mathbf{y}$ varies abouts its mean by using a weighted average of the variations of the features about their respective means!
2.  The optimal bias $\hat{b} \in \mathbb{R}$ is given by
\begin{equation*}
    \hat{b} = \overline{\mathbf{y}} - \sum_{i=1}^n \hat{w}_i \overline{\mathbf{x}}_i.
\end{equation*}
In other words, the fitted model must pass through the center of mass of the point-cloud in $\mathbb{R}^{n+1}$ defined by the dataset.

### Getting the matrix equation for the weights
Note that our original problem was to choose the parameters so that (1) holds. We used only the condition that $\mathbf{y} - \hat{\mathbf{y}}$ is orthogonal to $\mathbf{1}$ to show that the optimal weights are found by (6), and the optimal bias is then given by (4). 

Now, let us focus on solving (6). Observe that as $\mathbf{w}$ varies over $\mathbb{R}^n$, the vector $\sum_{i=1}^n w_i \mathbf{x}_i'$ varies over the subspace
\begin{equation*}
    U' = \textup{span}(\mathbf{x}_1',\dotsc,\mathbf{x}_n') \subseteq \mathbb{R}^m.
\end{equation*}
Thus, the optimal weights are found when $\sum_{i=1}^n w_i \mathbf{x}_i'$ is the closest point to $\mathbf{y}'$ in $U'$. This is equivalent to saying that $\sum_{i=1}^n w_i \mathbf{x}_i'$ is the orthogonal projection of $\mathbf{y}'$ onto $U'$. By definition, this means that $\sum_{i=1}^n w_i \mathbf{x}_i'$ is the unique vector in $U'$ such that
\begin{equation*}
    \mathbf{y}' - \sum_{i=1}^n w_i \mathbf{x}_i' \textup{ is orthogonal to every vector in } U'.
\end{equation*}
Like before, this is equivalent to saying that
\begin{equation}\tag{7}
    \mathbf{y}' - \sum_{i=1}^n w_i \mathbf{x}_i' \textup{ is orthogonal to } \mathbf{x}_1',\dotsc,\mathbf{x}_n'.
\end{equation}
The rest now proceeds exactly as in method 1. That is, (7) is equivalent to saying that
\begin{equation*}
    \mathbf{X}'^T (\mathbf{y}' - \mathbf{X}' \mathbf{w}) = 0.
\end{equation*}
We re-arrange and divide by $m$ on both sides to get:
\begin{equation}\tag{8}
    \frac{1}{m}\mathbf{X}'^T \mathbf{X}' \mathbf{w} = \frac{1}{m}\mathbf{X}'^T \mathbf{y}' \in \mathbb{R}^n.
\end{equation}
The reason for dividing by $m$ will become clear very soon. Indeed, observe that the right-hand side of (8) is
\begin{align*}
    \frac{1}{m}\mathbf{X}'^T \mathbf{y}' & = \frac{1}{m} \begin{bmatrix} \mathbf{x}_1' \cdot \mathbf{y}' \\ \vdots \\ \mathbf{x}_n' \cdot \mathbf{y}' \end{bmatrix} \\
                                        & = \begin{bmatrix} \frac{1}{m}(\mathbf{x}_1' \cdot \mathbf{y}') \\ \vdots \\ \frac{1}{m}(\mathbf{x}_n' \cdot \mathbf{y}') \end{bmatrix}\\
                                        & = \begin{bmatrix} \textup{cov}(\mathbf{x}_1,\mathbf{y}) \\ \vdots \\ \textup{cov}(\mathbf{x}_n,\mathbf{y}) \end{bmatrix} \in \mathbb{R}^n.
\end{align*}
Thus, the RHS of (8) is the vector of covariances between the features and the target!
The left-hand side is even prettier. The matrix $\frac{1}{m}\mathbf{X}'^T \mathbf{X}'$ is called the **covariance matrix** of the feature vectors $\mathbf{x}_1,\dotsc, \mathbf{x}_n$. It is denoted by $\Sigma$, and it satisfies:
\begin{align*}
    \frac{1}{m}\mathbf{X}'^T \mathbf{X}' & = \frac{1}{m} \begin{bmatrix} \mathbf{x}_1' \cdot \mathbf{x}_1' & \dotsb & \mathbf{x}_1' \cdot \mathbf{x}_n' \\ \vdots & \ddots & \vdots \\ \mathbf{x}_n' \cdot \mathbf{x}_1' & \dotsb & \mathbf{x}_n' \cdot \mathbf{x}_n' \end{bmatrix} \\
                                        & = \begin{bmatrix} \textup{cov}(\mathbf{x}_1,\mathbf{x}_1) & \dotsb & \textup{cov}(\mathbf{x}_1,\mathbf{x}_n) \\ \vdots & \ddots & \vdots \\ \textup{cov}(\mathbf{x}_n,\mathbf{x}_1) & \dotsb & \textup{cov}(\mathbf{x}_n,\mathbf{x}_n) \end{bmatrix} \in \mathbb{R}^{n \times n}.
\end{align*}
Note that the diagonal entries of $\Sigma$ are the variances $\textup{var}(\mathbf{x}_i) = \textup{cov}(\mathbf{x}_i,\mathbf{x}_i)$, and the off-diagonal entries are the covariances $\textup{cov}(\mathbf{x}_i,\mathbf{x}_j)$ between the distinct features!
Thus, the matrix equation (8) can be re-written as
\begin{equation}\tag{9}
    \Sigma \mathbf{w} = \begin{bmatrix} \textup{cov}(\mathbf{x}_1,\mathbf{x}_1) & \dotsb & \textup{cov}(\mathbf{x}_1,\mathbf{x}_n) \\ \vdots & \ddots & \vdots \\ \textup{cov}(\mathbf{x}_n,\mathbf{x}_1) & \dotsb & \textup{cov}(\mathbf{x}_n,\mathbf{x}_n) \end{bmatrix} \begin{bmatrix} w_1 \\ \vdots \\ w_n \end{bmatrix} = \begin{bmatrix} \textup{cov}(\mathbf{x}_1,\mathbf{y}) \\ \vdots \\ \textup{cov}(\mathbf{x}_n,\mathbf{y}) \end{bmatrix} = \mathbf{X}'^T \mathbf{y} \in \mathbb{R}^n.
\end{equation}
So, we conclude that the optimal weights $\hat{\mathbf{w}}$ are given by
\begin{equation}\tag{10}
    \hat{\mathbf{w}} = \Sigma^{-1} \mathbf{X}'^T \mathbf{y} = \begin{bmatrix} \textup{cov}(\mathbf{x}_1,\mathbf{x}_1) & \dotsb & \textup{cov}(\mathbf{x}_1,\mathbf{x}_n) \\ \vdots & \ddots & \vdots \\ \textup{cov}(\mathbf{x}_n,\mathbf{x}_1) & \dotsb & \textup{cov}(\mathbf{x}_n,\mathbf{x}_n) \end{bmatrix}^{-1} \begin{bmatrix} \textup{cov}(\mathbf{x}_1,\mathbf{y}) \\ \vdots \\ \textup{cov}(\mathbf{x}_n,\mathbf{y}) \end{bmatrix} \in \mathbb{R}^n.
\end{equation}
This concludes method 2 for fitting the model.